In [358]:
import os
import pandas as pd
pd.set_option('display.max_columns', 100)
pd.set_option('display.max_rows', 500)

In [359]:
df = pd.read_parquet(os.path.join('data','jizdenky.parquet'))

In [360]:
df = df[df['odkud'].str.contains('Brno') | df['odkud'].str.contains('Praha')]
df = df[df['kam'].str.contains('Brno') | df['kam'].str.contains('Praha')]

In [361]:
df.sample(5)

,oscrapovano,prodejce,odkud,kam,odjezd,predstih,predstih_d,predstih_h,cena,prostredek,volnych_mist,obsazenost,jizdni_doba,vzdalenost,zpozdeni,cena_poznamka,den,prestupy,vlaky,mistenka_zdarma,nahradni_bus,volna_mista_economy,volna_mista_economy_plus,volna_mista_economy_business,volna_mista_premium
7421,2024-11-01 14:33:53,ČD,Brno hl.n.,Praha hl.n.,2024-11-02 16:56:00,1 days 02:22:07,1,26.368611,329.0,vlak,NaN,[Předpokládané nízké obsazení],188.0,255.0,[0],None,so,0,[R 862 Svitava],True,None,NaN,NaN,NaN,NaN
416237,2024-11-14 19:11:27,RJ,Praha,Brno,2024-12-02 16:15:00,17 days 21:03:33,17,429.059167,299.0,vlak,323.0,None,151.0,NaN,None,None,po,0,None,None,None,NaN,NaN,NaN,NaN
414972,2024-11-14 18:54:30,ČD,Brno hl.n.,Praha hl.n.,2024-11-28 12:39:00,13 days 17:44:30,13,329.741667,359.0,vlak,NaN,[Předpokládané vysoké obsazení - doporučujeme ...,154.0,255.0,[0],None,čt,0,[rj 74 Vindobona],True,None,NaN,NaN,NaN,NaN
52373,2024-11-04 06:54:09,ČD,Praha hl.n.,Brno hl.n.,2024-11-05 05:44:00,0 days 22:49:51,0,22.830833,359.0,vlak,NaN,[Předpokládané nízké obsazení],155.0,255.0,[0],None,út,0,[EC 273 Metropolitan],True,None,NaN,NaN,NaN,NaN
183710,2024-11-09 00:10:17,RJ,Praha,Brno,2024-11-14 16:15:00,5 days 16:04:43,5,136.078611,399.0,vlak,168.0,None,151.0,NaN,None,None,čt,0,None,None,None,NaN,NaN,NaN,NaN


In [362]:
df = df[df['prestupy'] == 0]
# df['obsazenost'] = df['obsazenost'].apply(lambda x: x[0] if isinstance(x, list) else None)

In [363]:
df['jizdni_doba'].quantile([0.1,0.25,0.5,0.75,0.666,0.8,0.9])

0.100    148.0
0.250    152.0
0.500    155.0
0.750    186.0
0.666    179.0
0.800    188.0
0.900    224.0
Name: jizdni_doba, dtype: float64

In [364]:
nejdelsi = df['jizdni_doba'].quantile(0.666)
nejdelsi

179.0

In [365]:
df = df[df['jizdni_doba'] <= nejdelsi]

In [366]:
df = df[df['predstih_h'] <= 4]

In [367]:
df['spoj'] = df.apply(lambda row: f"""{row['prodejce']} {row['den']} {row['odjezd'].hour:02}:{row['odjezd'].minute:02}""", axis=1)

In [368]:
df['spoj'].sample(5)

393227    ČD čt 12:44
364095    ČD st 18:39
472734    RJ so 11:15
229171    ČD ne 05:39
373111    ČD st 19:44
Name: spoj, dtype: object

In [379]:
df.groupby('spoj')['odjezd'].nunique().sort_values()

spoj
RJ ne 06:15    1
RJ út 18:15    1
RJ út 18:09    1
RJ út 17:15    1
RJ út 16:15    1
RJ út 16:09    1
RJ út 15:15    1
RJ út 14:09    1
RJ út 13:15    1
RJ út 12:09    1
RJ út 11:15    1
RJ út 10:09    1
RJ út 09:15    1
RJ út 08:09    1
RJ út 07:15    1
RJ út 19:09    1
RJ út 07:09    1
RJ st 20:56    1
RJ st 19:09    1
RJ st 18:09    1
RJ st 16:15    1
RJ st 16:09    1
RJ st 15:15    1
RJ st 14:09    1
RJ st 13:15    1
RJ st 12:09    1
RJ st 11:15    1
RJ st 10:09    1
RJ st 09:15    1
RJ st 08:09    1
RJ st 07:15    1
RJ út 06:15    1
RJ út 20:56    1
RJ čt 06:15    1
RJ čt 07:09    1
ČD st 03:10    1
ČD so 04:42    1
ČD so 04:26    1
ČD pá 08:03    1
ČD pá 07:44    1
ČD pá 07:39    1
ČD pá 06:44    1
ČD pá 06:39    1
ČD pá 05:44    1
ČD pá 04:26    1
ČD po 04:42    1
ČD po 03:10    1
ČD po 00:40    1
ČD po 00:29    1
ČD po 00:03    1
RJ čt 20:56    1
RJ čt 19:09    1
RJ čt 07:15    1
RJ čt 08:09    1
RJ čt 09:15    1
RJ čt 10:09    1
RJ čt 11:15    1
RJ čt 12:09    1
RJ st 07:

In [369]:
df[df['predstih_h'] < 4].groupby('spoj')['volnych_mist'].min().nlargest(10)

spoj
RJ st 20:56    307.0
RJ út 20:56    296.0
RJ po 20:56    295.0
RJ pá 20:56    283.0
RJ so 20:56    238.0
RJ čt 20:56    227.0
RJ ne 06:15    220.0
RJ ne 07:09    219.0
RJ st 06:15    213.0
RJ út 18:15    208.0
Name: volnych_mist, dtype: float64

In [370]:
df[df['predstih_h'] < 4].groupby('spoj')['volnych_mist'].min().dropna().nsmallest(50)

spoj
RJ ne 11:15     0.0
RJ ne 13:15     0.0
RJ ne 14:09     0.0
RJ ne 15:15     0.0
RJ ne 16:09     0.0
RJ ne 16:15     0.0
RJ ne 17:15     0.0
RJ ne 18:09     0.0
RJ pá 10:09     0.0
RJ pá 12:09     0.0
RJ pá 14:09     0.0
RJ pá 15:15     0.0
RJ pá 16:09     0.0
RJ pá 16:15     0.0
RJ ne 19:09     1.0
RJ pá 08:09     1.0
RJ pá 09:15     1.0
RJ pá 13:15     1.0
RJ pá 17:15     2.0
RJ po 09:15     3.0
RJ so 10:09     3.0
RJ so 12:09     3.0
RJ čt 16:15     3.0
RJ čt 18:09     3.0
RJ ne 18:15     4.0
RJ čt 17:15     4.0
RJ ne 10:09     5.0
RJ čt 12:09     5.0
RJ so 09:15     6.0
RJ st 09:15     8.0
RJ so 08:09    10.0
RJ út 09:15    11.0
RJ čt 10:09    11.0
RJ so 11:15    12.0
RJ st 11:15    12.0
RJ čt 16:09    12.0
RJ pá 11:15    13.0
RJ st 12:09    13.0
RJ ne 12:09    14.0
RJ čt 07:15    14.0
RJ pá 07:15    15.0
RJ st 15:15    16.0
RJ so 14:09    17.0
RJ ne 09:15    18.0
RJ út 16:09    19.0
RJ čt 09:15    19.0
RJ st 16:15    22.0
RJ st 17:15    25.0
RJ čt 14:09    25.0
RJ so 07:15    

In [371]:
df[df['odkud'].str.contains('Brno')].groupby("spoj")['cena'].median().dropna().sort_values().head(20)

spoj
RJ út 19:09    179.0
RJ po 19:09    179.0
RJ ne 07:09    249.0
RJ út 18:09    249.0
RJ st 19:09    249.0
RJ st 18:09    249.0
RJ so 19:09    249.0
RJ so 18:09    249.0
RJ út 14:09    249.0
RJ po 18:09    249.0
RJ út 16:09    249.0
ČD út 21:39    269.0
ČD po 21:39    269.0
RJ čt 19:09    274.0
ČD st 03:10    295.0
ČD ne 03:10    295.0
RJ ne 08:09    299.0
RJ út 10:09    299.0
RJ út 08:09    299.0
RJ st 16:09    299.0
Name: cena, dtype: float64

In [372]:
df['cena'].median()

359.0

In [373]:
df[df['odkud'].str.contains('Praha')].groupby("spoj")['cena'].median().dropna().sort_values().head(20)

spoj
RJ čt 20:56     99.0
RJ út 20:56     99.0
RJ st 20:56     99.0
RJ po 20:56     99.0
RJ ne 06:15    179.0
RJ st 06:15    179.0
RJ út 06:15    179.0
RJ po 18:15    179.0
RJ so 18:15    249.0
RJ so 20:56    249.0
RJ st 07:15    249.0
RJ so 06:15    249.0
RJ pá 20:56    249.0
RJ st 18:15    249.0
RJ út 07:15    249.0
RJ út 11:15    249.0
RJ pá 06:15    249.0
RJ po 06:15    249.0
RJ ne 07:15    249.0
RJ čt 07:15    249.0
Name: cena, dtype: float64

In [374]:
df[df['odkud'].str.contains('Brno')].groupby("spoj")['cena'].median().dropna().sort_values().tail(20)

spoj
ČD ne 16:39    449.0
ČD pá 12:39    449.0
ČD ne 18:39    449.0
ČD pá 10:39    449.0
ČD pá 08:39    449.0
ČD pá 14:39    449.0
RJ čt 12:09    469.0
RJ pá 08:09    469.0
RJ ne 12:09    469.0
RJ so 10:09    469.0
RJ pá 12:09    469.0
RJ ne 18:09    469.0
ČD út 00:29    479.0
ČD po 00:29    479.0
ČD st 00:29    479.0
ČD čt 00:29    519.0
ČD pá 00:29    519.0
ČD so 00:29    519.0
ČD ne 00:29    519.0
RJ pá 10:09    524.0
Name: cena, dtype: float64

In [375]:
df[df['odkud'].str.contains('Praha')].groupby("spoj")['cena'].median().dropna().sort_values().tail(20)

spoj
ČD pá 09:44    465.0
RJ pá 09:15    469.0
RJ pá 17:15    469.0
RJ so 09:15    469.0
ČD pá 15:44    475.0
ČD pá 16:44    475.0
ČD pá 12:44    475.0
ČD pá 13:44    475.0
ČD pá 14:03    475.0
ČD pá 14:44    475.0
ČD po 00:40    479.0
RJ pá 15:15    489.0
RJ ne 17:15    499.0
ČD út 00:40    519.0
RJ pá 13:15    519.0
ČD st 00:40    519.0
ČD čt 00:40    519.0
ČD ne 00:40    519.0
ČD so 00:40    519.0
ČD pá 00:40    545.0
Name: cena, dtype: float64

In [376]:
df[df['prodejce'] == 'ČD'].groupby('spoj')['obsazenost'].agg(pd.Series.mode)

spoj
ČD ne 00:03                       [Předpokládané nízké obsazení]
ČD ne 00:29                       [Předpokládané nízké obsazení]
ČD ne 00:40                       [Předpokládané nízké obsazení]
ČD ne 03:10                       [Předpokládané nízké obsazení]
ČD ne 05:39                       [Předpokládané nízké obsazení]
ČD ne 05:44                       [Předpokládané nízké obsazení]
ČD ne 06:39                       [Předpokládané nízké obsazení]
ČD ne 06:44                       [Předpokládané nízké obsazení]
ČD ne 07:39                       [Předpokládané nízké obsazení]
ČD ne 07:44                       [Předpokládané nízké obsazení]
ČD ne 08:03                       [Předpokládané nízké obsazení]
ČD ne 08:39    [Předpokládané střední obsazení - doporučujeme...
ČD ne 08:44                       [Předpokládané nízké obsazení]
ČD ne 09:39    [[Předpokládané střední obsazení - doporučujem...
ČD ne 09:44    [Předpokládané vysoké obsazení - doporučujeme ...
ČD ne 10:03    [Před

In [377]:
df[df['prodejce'] == 'RJ'].groupby('spoj')['volnych_mist'].median()

spoj
RJ ne 06:15    220.0
RJ ne 07:09    219.0
RJ ne 07:15    183.0
RJ ne 08:09    110.0
RJ ne 09:15     18.5
RJ ne 10:09      5.5
RJ ne 11:15      1.0
RJ ne 12:09     14.0
RJ ne 13:15     10.0
RJ ne 14:09      0.0
RJ ne 15:15      0.0
RJ ne 16:09      0.0
RJ ne 16:15      0.0
RJ ne 17:15      0.5
RJ ne 18:09      1.0
RJ ne 18:15      4.0
RJ ne 19:09      2.5
RJ ne 20:56    149.0
RJ po 06:15     70.0
RJ po 07:09    171.0
RJ po 07:15     50.0
RJ po 08:09     76.0
RJ po 09:15      3.5
RJ po 10:09     80.0
RJ po 11:15     69.0
RJ po 12:09     61.5
RJ po 13:15    148.0
RJ po 14:09     54.0
RJ po 15:15    136.0
RJ po 18:09    124.0
RJ po 18:15    159.0
RJ po 19:09    210.5
RJ po 20:56    296.0
RJ pá 06:15     72.0
RJ pá 07:09     43.0
RJ pá 07:15     15.0
RJ pá 08:09      3.0
RJ pá 09:15      1.0
RJ pá 10:09      1.0
RJ pá 11:15     15.0
RJ pá 12:09      0.5
RJ pá 13:15      2.0
RJ pá 14:09      0.0
RJ pá 15:15      1.0
RJ pá 16:09      0.0
RJ pá 16:15      3.0
RJ pá 17:15      2.0
RJ pá 18

In [378]:
df.groupby('obsazenost').size()

TypeError: unhashable type: 'numpy.ndarray'

In [ ]:
df['spoj'].drop_duplicates().sort_values().to_list()

In [ ]:
df[df['prostredek'] == 'autobus']

In [ ]:
df = df[df['prostredek'] != 'autobus']

In [ ]:
def tyden(frejm):
    frame = frejm.copy()
    frame['hodina'] = frame['odjezd'].dt.hour
    frame['blok'] = frame['hodina'] // 4
    vysledek = frame.groupby(['den','blok'])['volnych_mist'].median().reset_index()
    bloky = {
        0: '00:00-03:59',
        1: '04:00-07:59',
        2: '08:00-11:59',
        3: '12:00-15:59',
        4: '16:00-19:59',
        5: '20:00-23:59'
    }
    poradi = {'po': 1, 'út': 2, 'st': 3, 'čt': 4, 'pá': 5, 'so': 6, 'ne': 7}
    vysledek['část_dne'] = vysledek['blok'].map(bloky)
    vysledek['poradi_dne'] = vysledek['den'].map(poradi).astype(int)
    vysledek = vysledek.sort_values(by=['poradi_dne','část_dne'], ascending=True)
    return vysledek[['den','část_dne','volnych_mist']]

In [ ]:
brnprg = tyden(df[(df['prodejce'] == 'RJ') & (df['predstih_h'] < 4) & (df['odkud'].str.contains('Brno'))])
prgbrn = tyden(df[(df['prodejce'] == 'RJ') & (df['predstih_h'] < 4) & (df['odkud'].str.contains('Praha'))])

In [ ]:
brnprg = brnprg.rename(columns={"volnych_mist": "brn->prg"})
prgbrn = prgbrn.rename(columns={"volnych_mist": "prg->brn"})

In [ ]:
presuny = brnprg.merge(prgbrn, left_on=['den','část_dne'], right_on=['den','část_dne'])

In [ ]:
presuny['rozdil'] = presuny['brn->prg'] - presuny['prg->brn']

In [ ]:
presuny['popis'] = presuny['den'].astype(str) + " " + presuny['část_dne']

In [ ]:
presuny

In [ ]:
presuny.set_index('popis')['rozdil'].plot.bar()